### 1. Metrics

In [3]:
import cirq
import numpy as np
from cirq.noise.utils.metrics import fidelity, operator_fidelity, trace_distance


#### 1.1 fidelity

In [4]:
# 设置测试数据
rho_ideal = np.array([[0.5, 0.5], [0.5, 0.5]])
rho_actual = np.array([[0.5, -0.5], [-0.5, 0.5]])

# fidelity 测试
fid = fidelity(rho_ideal, rho_ideal)
print(f"Fidelity between rho_ideal and rho_ideal: {fid}")  # 预期输出为1.0

fid_diff = fidelity(rho_ideal, rho_actual)
print(f"Fidelity between rho_ideal and rho_actual: {fid_diff}")  # 预期小于1.0


Fidelity between rho_ideal and rho_ideal: 1.0000000000000004
Fidelity between rho_ideal and rho_actual: 1.11022302266988e-16


In [5]:
# 零矩阵的保真度
zero_matrix = np.zeros_like(rho_ideal)
fid_zero = fidelity(zero_matrix, zero_matrix)
print(f"Fidelity between zero_matrix and zero_matrix: {fid_zero}")  # 预期输出为 0

# 单位矩阵和实际矩阵的保真度
rho_identity = np.eye(2) / 2
fid_id = fidelity(rho_identity, rho_ideal)
print(f"Fidelity between zero_matrix and rho_ideal: {fid_id}")  # 预期输出为 0

Fidelity between zero_matrix and zero_matrix: 0.0
Fidelity between zero_matrix and rho_ideal: 0.5000000000000002


#### 1.2 operator_fidelity (未使用)

#### 1.3 trace_distance

In [9]:
# 设置测试数据
rho_ideal = np.array([[0.5, 0.5], [0.5, 0.5]])  # 纯态密度矩阵
rho_actual = np.array([[0.5, -0.5], [-0.5, 0.5]])  # 混态密度矩阵

# 迹距离测试
trace_dist = trace_distance(rho_ideal, rho_ideal, type='eig')
print(f"Trace distance between rho_ideal and rho_ideal (eig method): {trace_dist}")  # 预计为正值

trace_dist_diff = trace_distance(rho_ideal, rho_actual, type='eig')
print(f"Trace distance between rho_ideal and rho_actual (eig method): {trace_dist_diff}")  # 预计为正值


# 使用 svd 方法计算
trace_dist_svd = trace_distance(rho_ideal, rho_ideal, type='svd')
print(f"Trace distance between rho_ideal and rho_ideal (svd method): {trace_dist_svd}")  # 预计为正值

trace_dist_svd_diff = trace_distance(rho_ideal, rho_actual, type='svd')
print(f"Trace distance between rho_ideal and rho_actual (svd method): {trace_dist_svd_diff}")  # 预计为正值


Trace distance between rho_ideal and rho_ideal (eig method): 0.0
Trace distance between rho_ideal and rho_actual (eig method): 1.0
Trace distance between rho_ideal and rho_ideal (svd method): 0.0
Trace distance between rho_ideal and rho_actual (svd method): 1.0


In [11]:
# 零矩阵和单位矩阵的迹距离
zero_matrix = np.zeros_like(rho_ideal)
trace_zero = trace_distance(zero_matrix, zero_matrix)
print(f"Trace distance between zero_matrix and zero_matrix: {trace_zero}")  # 预计为 0

# 单位矩阵和实际矩阵的迹距离
rho_identity = np.eye(2) / 2  # 单位矩阵
trace_identity = trace_distance(rho_identity, rho_actual)
print(f"Trace distance between rho_identity and rho_actual: {trace_identity}")  # 预计为正值


Trace distance between zero_matrix and zero_matrix: 0.0
Trace distance between rho_identity and rho_actual: 0.5


### 2. Compilation scheme

In [ ]:
import cirq
import numpy as np


from cirq.noise.utils.metrics import fidelity
from cirq.transformers import optimize_for_target_gateset
from cirq.noise.utils.compilation_scheme import CZ_Y_Z_gateset, merge_zpow_weak, merge_zpow_subcircuit, asapize_circuit
from cirq.noise.utils.circuit_factory import build_QSIH_circuits

gset = CZ_Y_Z_gateset()

q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)
print(circuit)

sim = cirq.DensityMatrixSimulator()
rho0 = sim.simulate(circuit).final_density_matrix

circuit_p1 = optimize_for_target_gateset(circuit, gateset=gset)
rho1 = sim.simulate(circuit_p1).final_density_matrix

circuit_p2 = merge_zpow_weak(circuit_p1)
rho2 = sim.simulate(circuit_p2).final_density_matrix

circuit_p3 = merge_zpow_subcircuit(circuit_p2)
rho3 = sim.simulate(circuit_p3).final_density_matrix

circuit_p4 = asapize_circuit(circuit_p3)
rho4 = sim.simulate(circuit_p4).final_density_matrix

print(f"circuit_p1's fidelity with original circuit:{fidelity(rho0, rho1)}\ncircuit_p2's fidelity with original circuit:{fidelity(rho0, rho2)}\ncircuit_p3's fidelity with original circuit:{fidelity(rho0, rho3)}\ncircuit_p4's fidelity with original circuit:{fidelity(rho0, rho4)}\n")

0: ───S───S───H───@───────
                  │
1: ───T───────────X───@───
                      │
2: ───────@───────────X───
          │
3: ───H───X───────────────
circuit_p1's fidelity with original circuit:1.0
circuit_p2's fidelity with original circuit:1.000000238418579
circuit_p3's fidelity with original circuit:1.000000238418579
circuit_p4's fidelity with original circuit:1.000000238418579



In [118]:
from supermarq.benchmarks import HamiltonianSimulation
bc = HamiltonianSimulation(num_qubits=6, time_step=1, total_time = 2)
circuit: cirq.Circuit = bc.circuit()[:-1]
print(circuit)
sim = cirq.DensityMatrixSimulator()
rho0 = sim.simulate(circuit).final_density_matrix

circuit_p1 = optimize_for_target_gateset(circuit, gateset=gset)
rho1 = sim.simulate(circuit_p1).final_density_matrix
circuit_p2 = merge_zpow_weak(circuit_p1)
rho2 = sim.simulate(circuit_p2).final_density_matrix

circuit_p3 = merge_zpow_subcircuit(circuit_p2)
rho3 = sim.simulate(circuit_p3).final_density_matrix

circuit_p4 = asapize_circuit(circuit_p3)
rho4 = sim.simulate(circuit_p4).final_density_matrix

print(f"circuit_p1's fidelity with original circuit:{fidelity(rho0, rho1)}\ncircuit_p2's fidelity with original circuit:{fidelity(rho0, rho2)}\ncircuit_p3's fidelity with original circuit:{fidelity(rho0, rho3)}\ncircuit_p4's fidelity with original circuit:{fidelity(rho0, rho4)}\n")

0: ───H───Rz(-0.75π)───H───@───────────────@───H───Rz(-0.749π)───H─────────────────────────@─────────────────@───────────────────────────────────────────────────────────────────────────────────────
                           │               │                                               │                 │
1: ───H───Rz(-0.75π)───H───X───Rz(-0.5π)───X───@─────────────────@───H───Rz(-0.749π)───H───X───Rz(-0.5π)─────X───@─────────────────@─────────────────────────────────────────────────────────────────
                                               │                 │                                               │                 │
2: ───H───Rz(-0.75π)───H───────────────────────X───Rz(-0.5π)─────X───@─────────────────@───H───Rz(-0.749π)───H───X───Rz(-0.5π)─────X───@─────────────────@───────────────────────────────────────────
                                                                     │                 │                                               │                 │
3: ───H

In [119]:
circuit_sym, circuit, _ = build_QSIH_circuits(n_qubits = 4, n_layers = 4, theta_value = 0.00)
print(circuit)
sim = cirq.DensityMatrixSimulator()
rho0 = sim.simulate(circuit).final_density_matrix

circuit_p1 = optimize_for_target_gateset(circuit, gateset=gset)
rho1 = sim.simulate(circuit_p1).final_density_matrix
print(circuit_p1)

circuit_p2 = merge_zpow_weak(circuit_p1)
rho2 = sim.simulate(circuit_p2).final_density_matrix

circuit_p3 = merge_zpow_subcircuit(circuit_p2)
rho3 = sim.simulate(circuit_p3).final_density_matrix

circuit_p4 = asapize_circuit(circuit_p3)
rho4 = sim.simulate(circuit_p4).final_density_matrix

print(f"circuit_p1's fidelity with original circuit:{fidelity(rho0, rho1)}\ncircuit_p2's fidelity with original circuit:{fidelity(rho0, rho2)}\ncircuit_p3's fidelity with original circuit:{fidelity(rho0, rho3)}\ncircuit_p4's fidelity with original circuit:{fidelity(rho0, rho4)}\n")

0: ───@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)───────────────────────────────────────────────────────@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)───────────────────────────────────────────────────────
      │                      │           │                                                                  │                      │           │
1: ───X───H───────Rz(0.5π)───X───H───────X───Rx(-0.5π)───@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)────X───H───────Rz(0.5π)───X───H───────X───Rx(-0.5π)───@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)────
                                                         │                      │           │                                                                  │                      │           │
2: ───@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)────X───H───────Rz(0.5π)───X───H───────X───Rx(-0.5π)───@───Rz(0)──────────────@───Rz(0)───@───Rx(0.5π)────X───H───────Rz(0.5π)───X───H───────X───Rx(-0.5π)───
      │                   

In [123]:
cirq.fidelity(rho0, rho1, [16], atol=1e-8)

np.float32(1.0011023)

### 3.Noise models

In [1]:
import cirq
import numpy as np

from cirq.noise.models import IdleNoiseModel, PhotonDecayNoiseModel, RyGateNoiseModel, SegmentedFluxNoiseModel, sample_flux_noise_segments
from cirq.noise.utils.composite_noise_model import CompositeNoiseModel
from cirq.noise.utils.compilation_scheme import all_in_one_compile
from cirq.noise.utils.metrics import fidelity, trace_distance
from cirq.noise.utils.timed_circuit_context import assign_timed_circuit_context

q0, q1, q2, q3 = cirq.LineQubit.range(4)
circuit0 = cirq.Circuit(
    cirq.S(q0),
    cirq.S(q0),
    cirq.T(q1),
    cirq.H(q0),
    cirq.H(q3),
    cirq.CX(q0, q1),
    cirq.CX(q2, q3),
    cirq.CX(q1, q2),
)
circuit = all_in_one_compile(circuit0)
print(circuit)

sim = cirq.DensityMatrixSimulator()
rho_ideal = sim.simulate(circuit).final_density_matrix
rho0 = sim.simulate(circuit0).final_density_matrix

                                     [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5─── ]─────────[       │    ]───────[ 0: ───Z^0.324───Y───Z^-0.676─── ]─────────────────────────────────────────────────────
                                     [ 1: ───@─── ]
                                     │
                                     │                                                          [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5─── ]───#2───────────────────[ 1: ───Y^0.5─── ]────────────────────[       │        ]────────────────────────────────
                                                                                                [ 2: ───@─────── ]
                                                                                                │
      [ 2: ───@─── ]                                                                            │
2: ───[       │    ]─────────────────[ 2: ───Y^0.5─── ]─────────────────────────────────────────#2───────────────────[ 2: ───Z───Y^0.5───

#### 3.0 

#### 3.1 cirq.noise.models.IdleNoiseModel

In [4]:
circuit_with_noise_idle = circuit.with_noise(IdleNoiseModel(
    t1=00e-9,
    t2=00e-9,
    T1=30e-6,
    Tphi=60e-6,
    scale=1,
))
rho_idle = sim.simulate(circuit_with_noise_idle).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_idle}\nfidelity compared with original circuit:{fidelity(rho0,rho_idle)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal,rho_idle)}")

circuit_with_noise_idle:
                                                     [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5───AD(0)───PD(0)─── ]─────────[       │    ]───────────────────────[ 0: ───Z^0.324───Y───Z^-0.676─── ]───[ 0: ───AD(0)───PD(0)─── ]───[ 0: ───AD(0)───PD(0)─── ]───
                                                     [ 1: ───@─── ]
                                                     │
                                                     │                                                                          [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───AD(0)───PD(0)─── ]───#2───────────────────────────────────[ 1: ───Y^0.5─── ]────────────────────[       │        ]───────────[ 1: ───AD(0)───PD(0)─── ]───
                                                                                                                                [ 2: ───@─────── ]
                                                                                                                    

In [8]:
circuit_with_noise_idle = circuit.with_noise(IdleNoiseModel(
    t1=20e-9,
    t2=40e-9,
    T1=30e-6,
    Tphi=60e-6,
    scale=1,
))
rho_idle = sim.simulate(circuit_with_noise_idle).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_idle}\nfidelity compared with original circuit:{fidelity(rho0,rho_idle)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal,rho_idle)}")

circuit_with_noise_idle:
                                                                   [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5───AD(0.000666)───PD(0.000333)─── ]─────────[       │    ]─────────────────────────────────────[ 0: ───Z^0.324───Y───Z^-0.676─── ]────────[ 0: ───AD(0.00133)───PD(0.000666)─── ]───[ 0: ───AD(0.000666)───PD(0.000333)─── ]───
                                                                   [ 1: ───@─── ]
                                                                   │
                                                                   │                                                                                             [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───AD(0.000666)───PD(0.000333)─── ]───#2─────────────────────────────────────────────────[ 1: ───Y^0.5─── ]─────────────────────────[       │        ]────────────────────────[ 1: ───AD(0.000666)───PD(0.000333)─── ]───
                                                                    

In [9]:
circuit_with_noise_idle = circuit.with_noise(IdleNoiseModel(
    t1=20e-9,
    t2=40e-9,
    T1=10e-6,
    Tphi=20e-6,
    scale=2,
))
rho_idle = sim.simulate(circuit_with_noise_idle).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_idle}\nfidelity compared with original circuit:{fidelity(rho0,rho_idle)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal,rho_idle)}")

circuit_with_noise_idle:
                                                               [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5───AD(0.00399)───PD(0.002)─── ]─────────[       │    ]─────────────────────────────────[ 0: ───Z^0.324───Y───Z^-0.676─── ]────[ 0: ───AD(0.00797)───PD(0.00399)─── ]───[ 0: ───AD(0.00399)───PD(0.002)─── ]───
                                                               [ 1: ───@─── ]
                                                               │
                                                               │                                                                                     [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───AD(0.00399)───PD(0.002)─── ]───#2─────────────────────────────────────────────[ 1: ───Y^0.5─── ]─────────────────────[       │        ]───────────────────────[ 1: ───AD(0.00399)───PD(0.002)─── ]───
                                                                                                                              

#### 3.2 cirq.noise.models.PhotonDecayNoiseModel

In [10]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 60,
    treat_z_as_virtual = True
    )

circuit_with_noise_photon = circuit.with_noise(PhotonDecayNoiseModel(
    ctx = ctx,
    chi = np.pi * (-2.6e6),
    alpha_0 = 0,
    kappa = 4e6,
    tau_m = -590e-9,
    tau_g = 10e-9,
    scale = 1
))
rho_photon = sim.simulate(circuit_with_noise_photon).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_idle}\nfidelity compared with original circuit:{fidelity(rho0, rho_photon)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_photon)}")

circuit_with_noise_idle:
                                                     [ 0: ───@─── ]
0: ───[ 0: ───Z───Y^0.5───AD(0)───PD(0)─── ]─────────[       │    ]───────────────────────[ 0: ───Z^0.324───Y───Z^-0.676─── ]───[ 0: ───AD(0)───PD(0)─── ]───[ 0: ───AD(0)───PD(0)─── ]───
                                                     [ 1: ───@─── ]
                                                     │
                                                     │                                                                          [ 1: ───@───Z─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───AD(0)───PD(0)─── ]───#2───────────────────────────────────[ 1: ───Y^0.5─── ]────────────────────[       │        ]───────────[ 1: ───AD(0)───PD(0)─── ]───
                                                                                                                                [ 2: ───@─────── ]
                                                                                                                    

In [7]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 60,
    treat_z_as_virtual = True
    )

circuit_with_noise_photon = circuit.with_noise(PhotonDecayNoiseModel(
    ctx = ctx,
    chi = np.pi * (-2.6e6),
    alpha_0 = np.sqrt(0.8),
    kappa = 4e6,
    tau_m = -590e-9,
    tau_g = 10e-9,
    scale = 1
))
rho_photon = sim.simulate(circuit_with_noise_photon).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_photon}\nfidelity compared with original circuit:{fidelity(rho0, rho_photon)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_photon)}")

circuit_with_noise_idle:
                                                  [ 0: ───@───PD(0.0261)─── ]
0: ───[ 0: ───Z───Y^0.5───PD(0.0212)─── ]─────────[       │                 ]───────[ 0: ───Z^0.324───Y───Z^-0.676───PD(0.0266)─── ]───[ 0: ───PD(0.0211)─── ]────────────────────[ 0: ───PD(0.00517)─── ]───────────────────
                                                  [ 1: ───@───PD(0.0261)─── ]
                                                  │
                                                  │                                                                                    [ 1: ───@───Z────────────PD(0.0211)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───PD(0.0212)─── ]───#2────────────────────────────────[ 1: ───Y^0.5───PD(0.0266)─── ]────────────────────[       │                              ]───[ 1: ───PD(0.00517)─── ]───────────────────
                                                                                                                                       [ 2: ───@───

In [8]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 60,
    treat_z_as_virtual = True
    )

circuit_with_noise_photon = circuit.with_noise(PhotonDecayNoiseModel(
    ctx = ctx,
    chi = np.pi * (-2.6e6),
    alpha_0 = np.sqrt(0.8),
    kappa = 4e6,
    tau_m = -590e-9,
    tau_g = 10e-9,
    scale = 3
))
rho_photon = sim.simulate(circuit_with_noise_photon).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_photon}\nfidelity compared with original circuit:{fidelity(rho0, rho_photon)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_photon)}")

circuit_with_noise_idle:
                                                  [ 0: ───@───PD(0.0763)─── ]
0: ───[ 0: ───Z───Y^0.5───PD(0.0622)─── ]─────────[       │                 ]───────[ 0: ───Z^0.324───Y───Z^-0.676───PD(0.0776)─── ]───[ 0: ───PD(0.062)─── ]───────────────────[ 0: ───PD(0.0154)─── ]───────────────────
                                                  [ 1: ───@───PD(0.0763)─── ]
                                                  │
                                                  │                                                                                    [ 1: ───@───Z───────────PD(0.062)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───PD(0.0622)─── ]───#2────────────────────────────────[ 1: ───Y^0.5───PD(0.0776)─── ]────────────────────[       │                            ]───[ 1: ───PD(0.0154)─── ]───────────────────
                                                                                                                                       [ 2: ───@───PD(0.062

#### 3.3 cirq.noise.models.RyGateNoiseModel

In [ ]:
circuit_with_noise_Ry = circuit.with_noise(RyGateNoiseModel(
    t1 = 0e-9,
    t2 = 0e-9,
    T1 = 30e-6,
    Tphi = 60e-6,
    p_axis = 1e-4,
    p_plane = 5e-4,
    scale = 1
))
rho_Ry = sim.simulate(circuit_with_noise_Ry).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_Ry}\nfidelity compared with original circuit:{fidelity(rho0,rho_Ry)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal,rho_Ry)}")

circuit_with_noise_idle:
                                                                                                       [ 0: ───AD(0)───PD(0)───@───AD(0)───PD(0)─── ]
0: ───[ 0: ───AD(0)───PD(0)───Z───Y^0.5───Dep(axis=Y,a=1.0e-04,p=5.0e-04)───AD(0)───PD(0)─── ]─────────[                       │                    ]─────────────────────────────────────────[ 0: ───AD(0)───PD(0)───Z^0.324───Y───Z^-0.676───Dep(axis=Y,a=1.0e-04,p=5.0e-04)───AD(0)───PD(0)─── ]───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                       [ 1: ───AD(0)───PD(0)───@───AD(0)───PD(0)─── ]
                                                                                                       │
                                                                                                       │                   

In [ ]:
circuit_with_noise_Ry = circuit.with_noise(RyGateNoiseModel(
    t1 = 20e-9,
    t2 = 40e-9,
    T1 = 30e-6,
    Tphi = 60e-6,
    p_axis = 1e-4,
    p_plane = 5e-4,
    scale = 1
))
rho_Ry = sim.simulate(circuit_with_noise_Ry).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_Ry}\nfidelity compared with original circuit:{fidelity(rho0,rho_Ry)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_Ry)}")

circuit_with_noise_idle:
                                                                                                                                   [ 0: ───AD(0.000666)───PD(0.000333)───@───AD(0.000666)───PD(0.000333)─── ]
0: ───[ 0: ───AD(0.000333)───PD(0.000167)───Z───Y^0.5───Dep(axis=Y,a=1.0e-04,p=5.0e-04)───AD(0.000333)───PD(0.000167)─── ]─────────[                                     │                                  ]─────────────────────────────────────────[ 0: ───AD(0.000333)───PD(0.000167)───Z^0.324───Y───Z^-0.676───Dep(axis=Y,a=1.0e-04,p=5.0e-04)───AD(0.000333)───PD(0.000167)─── ]──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                   [ 1: ───AD(0.000666)───PD(0.000333)───@───AD

In [ ]:
circuit_with_noise_Ry = circuit.with_noise(RyGateNoiseModel(
    t1 = 20e-9,
    t2 = 40e-9,
    T1 = 30e-6,
    Tphi = 60e-6,
    p_axis = 10e-4,
    p_plane = 50e-4,
    scale = 1
))
rho_Ry = sim.simulate(circuit_with_noise_Ry).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_Ry}\nfidelity compared with original circuit:{fidelity(rho0, rho_Ry)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_Ry)}")

circuit_with_noise_idle:
                                                                                                                                   [ 0: ───AD(0.000666)───PD(0.000333)───@───AD(0.000666)───PD(0.000333)─── ]
0: ───[ 0: ───AD(0.000333)───PD(0.000167)───Z───Y^0.5───Dep(axis=Y,a=1.0e-03,p=5.0e-03)───AD(0.000333)───PD(0.000167)─── ]─────────[                                     │                                  ]─────────────────────────────────────────[ 0: ───AD(0.000333)───PD(0.000167)───Z^0.324───Y───Z^-0.676───Dep(axis=Y,a=1.0e-03,p=5.0e-03)───AD(0.000333)───PD(0.000167)─── ]──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                   [ 1: ───AD(0.000666)───PD(0.000333)───@───AD

#### 3.4 cirq.noise.models.SegmentedFluxNoiseModel

In [8]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 60,
    treat_z_as_virtual = True
    )

max_seg = max(info.segment_id for infos in ctx.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(
    qubits = sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))),
    max_segment = max_seg,
    sigma = 0.00
    )

circuit_with_noise_flux = circuit.with_noise(SegmentedFluxNoiseModel(
    timing_map = ctx.timing_map,
    delta_phis = delta_phis,
    ))
rho_flux = sim.simulate(circuit_with_noise_flux).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_flux}\nfidelity compared with original circuit:{fidelity(rho0, rho_flux)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_flux)}")

circuit_with_noise_idle:
                                                     [ 0: ───@───eD(0.00e+00)───eE(0.00e+00)─── ]
0: ───[ 0: ───Z───Y^0.5───eD(0.00e+00)─── ]──────────[       │                  │               ]───[ 0: ───Z^0.324───Y───Z^-0.676───eD(0.00e+00)─── ]──────────────────────────────────────────────────────────────────────────────────────────────────
                                                     [ 1: ───@──────────────────eE(0.00e+00)─── ]
                                                     │
                                                     │                                                                                                   [ 1: ───@───Z───eD(0.00e+00)───eE(0.00e+00)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───eD(0.00e+00)─── ]────#2─────────────────────────────────────────────[ 1: ───Y^0.5───eD(0.00e+00)─── ]────────────────────[       │                      │               ]───────────────────────────────────────────────
                        

In [3]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 80,
    treat_z_as_virtual = True
    )

max_seg = max(info.segment_id for infos in ctx.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(
    qubits = sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))),
    max_segment = max_seg,
    sigma = 0.05
    )

circuit_with_noise_flux = circuit.with_noise(SegmentedFluxNoiseModel(
    timing_map = ctx.timing_map,
    delta_phis = delta_phis,
    ))
rho_flux = sim.simulate(circuit_with_noise_flux).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_flux}\nfidelity compared with original circuit:{fidelity(rho0, rho_flux)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_flux)}")

circuit_with_noise_idle:
                                                     [ 0: ───@───eD(2.16e-02)───eE(-1.01e-02)─── ]
0: ───[ 0: ───Z───Y^0.5───eD(2.16e-02)─── ]──────────[       │                  │                ]───[ 0: ───Z^0.324───Y───Z^-0.676───eD(1.24e-02)─── ]──────────────────────────────────────────────────────────────────────────────────────────────────
                                                     [ 1: ───@──────────────────eE(-1.01e-02)─── ]
                                                     │
                                                     │                                                                                                    [ 1: ───@───Z───eD(1.36e-02)───eE(4.29e-03)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───eD(-1.01e-02)─── ]───#2──────────────────────────────────────────────[ 1: ───Y^0.5───eD(1.36e-02)─── ]────────────────────[       │                      │               ]───────────────────────────────────────────────
                   

In [10]:
ctx = assign_timed_circuit_context(
    circuit = circuit,
    t1 = 20,
    t2 = 40,
    tau_c = 60,
    treat_z_as_virtual = True
    )

max_seg = max(info.segment_id for infos in ctx.timing_map.values() for info in infos)
delta_phis = sample_flux_noise_segments(
    qubits = sorted(circuit.all_qubits(), key=lambda q: getattr(q, "x", str(q))),
    max_segment = max_seg,
    sigma = 0.5
    )

circuit_with_noise_flux = circuit.with_noise(SegmentedFluxNoiseModel(
    timing_map = ctx.timing_map,
    delta_phis = delta_phis,
    ))
rho_flux = sim.simulate(circuit_with_noise_flux).final_density_matrix

print(f"circuit_with_noise_idle:\n{circuit_with_noise_flux}\nfidelity compared with original circuit:{fidelity(rho0, rho_flux)}")
print(f"fidelity compared with compiled circuit:{fidelity(rho_ideal, rho_flux)}")

circuit_with_noise_idle:
                                                     [ 0: ───@───eD(-2.44e-02)───eE(3.50e-03)─── ]
0: ───[ 0: ───Z───Y^0.5───eD(-2.44e-02)─── ]─────────[       │                   │               ]───[ 0: ───Z^0.324───Y───Z^-0.676───eD(7.50e-01)─── ]────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     [ 1: ───@───────────────────eE(3.50e-03)─── ]
                                                     │
                                                     │                                                                                                    [ 1: ───@───Z───eD(-4.41e-01)───eE(-6.54e-01)─── ]
1: ───[ 1: ───Z^-0.75───Y^0.5───eD(3.50e-03)─── ]────#2──────────────────────────────────────────────[ 1: ───Y^0.5───eD(-4.41e-01)─── ]───────────────────[       │                       │                ]───────────────────────────────────────────────
             

#### 3.5 cirq.noise.models.CompositeNoiseModel